In [0]:
!pip install tensorflow-gpu==2.1

     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 450kB 39.7MB/s 
     |████████████████████████████████| 3.8MB 43.3MB/s 
     |████████████████████████████████| 81kB 14.8MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled goo

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!tar -xf "./drive/My Drive/Datasets/fer_4_classes.tar.xz"

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import csv

import tensorflow as tf
from tensorflow import keras
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.version.VERSION)

2.1.0


In [0]:
BATCH_SIZE = 128
IMG_HEIGHT = 160
IMG_WIDTH = 160
IMG_SIZE = 160
train_dir = "./4 classes fer/train"
test_dir = "./4 classes fer/test"
valid_dir = "./4 classes fer/valid"
CLASS_NAMES = np.array(['anger', 'happiness', 'neutral', 'sadness'])

In [0]:
data_generator = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_data_generator = ImageDataGenerator(rescale = 1./255.)

In [0]:
train_data_gen = data_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))
valid_data_gen = test_data_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=valid_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))
test_data_gen = test_data_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=test_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

Found 23814 images belonging to 4 classes.
Found 2968 images belonging to 4 classes.
Found 2953 images belonging to 4 classes.


In [0]:
faceNet = tf.keras.models.load_model('./drive/My Drive/Models/keras-facenet/model/facenet_keras.h5')

In [0]:
faceNet.trainable = True
for layer in faceNet.layers[:len(faceNet.layers)-2]:
  layer.trainable = False
faceNet.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [0]:
model = tf.keras.Sequential([
  faceNet,
  keras.layers.Dropout(0.3),
  keras.layers.Dense(4, activation="softmax")
])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v1 (Model)  (None, 128)               22808144  
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 4)                 516       
Total params: 22,808,660
Trainable params: 230,020
Non-trainable params: 22,578,640
_________________________________________________________________


In [0]:
num_train = 23814
num_test = 2968
num_valid = 2953
steps_per_epoch = round(num_train)//BATCH_SIZE
validation_steps = round(num_valid)//BATCH_SIZE

In [0]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=steps_per_epoch*1000,
  decay_rate=1,
  staircase=False)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [0]:
model.evaluate(valid_data_gen)
model.evaluate(test_data_gen)

  ...
    to  
  ['...']
24/24 [==============================] - 14s 577ms/step - loss: 2.4080 - accuracy: 0.1779
  ...
    to  
  ['...']
24/24 [==============================] - 4s 160ms/step - loss: 2.3756 - accuracy: 0.1693


[2.3755536327759423, 0.16931933]

In [0]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True)

history = model.fit(train_data_gen,
                    callbacks=[callback],
                    steps_per_epoch=steps_per_epoch,
                    epochs=100,
                    validation_data=valid_data_gen,
                    validation_steps=validation_steps)
model.save("./drive/My Drive/Models/facenet2.0.1")
model.evaluate(test_data_gen)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 186 steps, validate for 23 steps
Epoch 1/100
186/186 [==============================] - 154s 828ms/step - loss: 1.2540 - accuracy: 0.4824 - val_loss: 1.0019 - val_accuracy: 0.5414
Epoch 2/100
186/186 [==============================] - 150s 804ms/step - loss: 1.0356 - accuracy: 0.5686 - val_loss: 0.8776 - val_accuracy: 0.6478
Epoch 3/100
186/186 [==============================] - 154s 825ms/step - loss: 0.9657 - accuracy: 0.6009 - val_loss: 0.8524 - val_accuracy: 0.6389
Epoch 4/100
186/186 [==============================] - 148s 798ms/step - loss: 0.9302 - accuracy: 0.6199 - val_loss: 0.8398 - val_accuracy: 0.6484
Epoch 5/100
186/186 [==============================] - 148s 797ms/step - loss: 0.9054 - accuracy: 0.6263 - val_loss: 0.8395 - val_accuracy: 0.6403
Epoch 6/100
186/186 [==============================] - 149s 802ms/step - loss: 0.8983 - accuracy: 0.6331 - val_loss: 0.8498 - val_accuracy: 0.6362
Epoch 7/100
186/186 [====

[0.8270260492960612, 0.6657636]